In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# 정답 레이블 생성

In [ ]:
import os

fDir = '/content/drive/MyDrive/ComputerProgramming/DeepLearning/2.딥러닝/res/dog_cat_data/Images'

labels = list()
for dir in os.listdir(fDir):
  dog_type = '-'.join(dir.split('-')[1:])
  for f in os.listdir( os.path.join( fDir, dir) ):
    name = f.split('.')[0]
    data = f'{dir},{name},{dog_type}\n'
    labels.append( data )

with open('new_labels.csv', 'w') as f:
  f.write( 'dir,name,spec\n'+ ''.join(labels) )

# 훈련 데이터 로드

In [ ]:
import cv2
import pandas as pd

In [ ]:
labels    = pd.read_csv('/content/new_labels.csv')

In [ ]:
base_path = '/content/drive/MyDrive/ComputerProgramming/DeepLearning/2.딥러닝/res/dog_cat_data/Images'
train_X   = list()
for i in range(100):
  dir   = labels.loc[i, 'dir']
  name  = labels.loc[i, 'name']
  fName = f'{base_path}/{dir}/{name}.jpg'
  img   = cv2.imread( fName )
  img   = cv2.resize( img, dsize=(224,224) )
  img   = img / 255.0
  train_X.append( img )

In [ ]:
import numpy as np
train_X = np.array( train_X )
train_X.shape

(100, 224, 224, 3)

In [ ]:
unique_label = labels.spec.unique().tolist()
train_y      = [ unique_label.index(name) for name in labels.spec ]
train_y      = np.array( train_y )
train_y.shape

(5822,)

In [ ]:
train_y[:2]

array([0, 0])

# 전이학습

- 사전 학습된 모델 로드
- **파인 튜닝 전략**
  - 기존의 모델이 가진 가중치를 활용
    - 마지막 30개의 레이어의 가중치는 버린다.
    - 새로운 데이터를 보내서 학습을 진행하는데 마지막 30개만 훈련하도록 진행
      - 상위 126개는 훈련 X
      - 하위 30개 훈련 O
    - 마지막 출력은 120개로 수렴하도록 수정

## 사전 학습된 모델 가져오기

In [ ]:
from tensorflow.keras.applications import MobileNetV2
mobilev2 = MobileNetV2()
# (?, 224,224,3) => (?, 1000), 156 layers

In [ ]:
import tensorflow as tf
# 출력층 교체 (?, 1000) => (?, 120)
x        = mobilev2.layers[-2].output
preModel = tf.keras.layers.Dense( 120, activation='softmax' )( x )

In [ ]:
# 신경망 교체 완료
model = tf.keras.Model( inputs=mobilev2.input, outputs=preModel )

In [ ]:
# 학습 여부를 층별로 조정 -> 하위 30개만 학습 진행하겟다
for layer in mobilev2.layers[:-30]:
  layer.trainable = False # 상위 126개는 가중치를 유지한다. 학습영향없다
for layer in mobilev2.layers[-30:]:
  layer.trainable = True  # 하위 30개는 가중치를 새로 학습해서 조정한다

In [ ]:
# 컴파일
model.compile( optimizer='sgd', loss='sparse_categorical_crossentropy', 
               metrics=['accuracy'])
'''
Total params: 2,411,704
Trainable params: 1,526,200     <- 학습 대상 파라미터
Non-trainable params: 885,504
'''
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 Conv1 (Conv2D)                 (None, 112, 112, 32  864         ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 bn_Conv1 (BatchNormalization)  (None, 112, 112, 32  128         ['Conv1[0][0]']                  
                                )                                                             

In [ ]:
hist = model.fit( train_X, train_y[:100], epochs=10, 
                 validation_split=0.25, batch_size=32 )
# 데이터 셔플을 통해서 학습 진행 요망

Epoch 1/10
3/3 [==============================] - 13s 2s/step - loss: 3.7337 - accuracy: 0.3467 - val_loss: 0.7768 - val_accuracy: 1.0000
Epoch 2/10
3/3 [==============================] - 4s 1s/step - loss: 0.3490 - accuracy: 1.0000 - val_loss: 0.2953 - val_accuracy: 1.0000
Epoch 3/10
3/3 [==============================] - 4s 1s/step - loss: 0.1058 - accuracy: 1.0000 - val_loss: 0.1954 - val_accuracy: 1.0000
Epoch 4/10
3/3 [==============================] - 4s 1s/step - loss: 0.0605 - accuracy: 1.0000 - val_loss: 0.1503 - val_accuracy: 1.0000
Epoch 5/10
3/3 [==============================] - 4s 1s/step - loss: 0.0414 - accuracy: 1.0000 - val_loss: 0.1214 - val_accuracy: 1.0000
Epoch 6/10
3/3 [==============================] - 4s 1s/step - loss: 0.0322 - accuracy: 1.0000 - val_loss: 0.1028 - val_accuracy: 1.0000
Epoch 7/10
3/3 [==============================] - 4s 1s/step - loss: 0.0253 - accuracy: 1.0000 - val_loss: 0.0904 - val_accuracy: 1.0000
Epoch 8/10
3/3 [========================